# 크롤링(Crawling, Scraping)
웹페이지에서 필요한 자료 추출

## 기상청 크롤링

In [1]:
url = 'http://www.kma.go.kr/wid/queryDFSRSS.jsp?zone=1168064000'
import requests
res = requests.get(url)
# res.text

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
from bs4 import BeautifulSoup
bs = BeautifulSoup(res.text,'html.parser')
# bs.text

In [3]:
# bs.select('') 모든 태그 -> 리스트
# bs.select_one('') 한개 태그
bs.select_one('category').string

'서울특별시 강남구 역삼1동'

In [4]:
#일기예보 데이터 추출 (3시간)
data_list = bs.select('data')
print(f'갯수:{len(data_list)}')
data_arr = []
day_arr = ['오늘','내일','모레']
for dt in data_list:
    data_dict = dict()
    data_dict['시간'] = dt.select_one('hour').string
    data_dict['날짜'] = day_arr[int(dt.select_one('day').string)]
    data_dict['날씨'] = dt.select_one('wfkor').string
    data_dict['온도'] = dt.select_one('temp').string
    data_dict['강수'] = dt.select_one('pop').string
    data_dict['습도'] = dt.select_one('reh').string
    data_arr.append(data_dict)    
data_arr

갯수:17


[{'시간': '24', '날짜': '오늘', '날씨': '맑음', '온도': '23.0', '강수': '0', '습도': '75'},
 {'시간': '3', '날짜': '내일', '날씨': '맑음', '온도': '21.0', '강수': '0', '습도': '80'},
 {'시간': '6', '날짜': '내일', '날씨': '맑음', '온도': '21.0', '강수': '0', '습도': '80'},
 {'시간': '9', '날짜': '내일', '날씨': '맑음', '온도': '24.0', '강수': '0', '습도': '70'},
 {'시간': '12', '날짜': '내일', '날씨': '맑음', '온도': '26.0', '강수': '10', '습도': '65'},
 {'시간': '15', '날짜': '내일', '날씨': '구름 많음', '온도': '27.0', '강수': '10', '습도': '60'},
 {'시간': '18', '날짜': '내일', '날씨': '흐림', '온도': '25.0', '강수': '30', '습도': '70'},
 {'시간': '21', '날짜': '내일', '날씨': '흐림', '온도': '24.0', '강수': '30', '습도': '75'},
 {'시간': '24', '날짜': '내일', '날씨': '흐림', '온도': '23.0', '강수': '30', '습도': '75'},
 {'시간': '3', '날짜': '모레', '날씨': '흐림', '온도': '23.0', '강수': '30', '습도': '80'},
 {'시간': '6', '날짜': '모레', '날씨': '흐림', '온도': '23.0', '강수': '30', '습도': '80'},
 {'시간': '9', '날짜': '모레', '날씨': '비', '온도': '24.0', '강수': '61', '습도': '85'},
 {'시간': '12', '날짜': '모레', '날씨': '비', '온도': '26.0', '강수': '77', '습도': '85'},
 {'시간': 

</br>
</br>

## KOVO 경기결과 크롤링

In [5]:
import requests
from bs4 import BeautifulSoup
url = 'https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp'
res = requests.get(url,verify = False)
bs = BeautifulSoup(res.text,'html.parser')

C:\Users\admin\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.kovo.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [6]:
#코보 데이터 추출

tbl = bs.select_one('table.lst_schedule')
tbl_tr = tbl.select('tr')
data_list=''
date = ''
for tr in tbl_tr:
    if len(tr.select('td.tright')) == 0:
        continue
    if tr.select_one('td.date').get_text() != '':
        date = tr.select_one('td.date').get_text()
    game_result = []
    game_result.append(date)
    tmp = tr.select_one('td.tright').get_text().replace(u'\xa0', u' ')
    tmp += tr.select_one('td.tleft').get_text().replace(u'\xa0', u' ')
    tmp = tmp.split(" ")
    for i in range(len(tmp)):
        if tmp[i] !='':
            game_result.append(tmp[i])
    game_result=','.join(game_result)
    data_list+=game_result+'\n'
data_list

'10. 16 (토),대한항공,3,:,1,우리카드\n10. 16 (토),GS칼텍스,3,:,0,흥국생명\n10. 17 (일),현대캐피탈,3,:,1,OK금융그룹\n10. 17 (일),현대건설,3,:,1,IBK기업은행\n10. 19 (화),삼성화재,0,:,3,한국전력\n10. 19 (화),페퍼저축은행,1,:,3,KGC인삼공사\n10. 20 (수),KB손해보험,3,:,2,현대캐피탈\n10. 20 (수),한국도로공사,0,:,3,현대건설\n10. 21 (목),OK금융그룹,3,:,2,우리카드\n10. 21 (목),IBK기업은행,1,:,3,흥국생명\n10. 22 (금),삼성화재,3,:,0,대한항공\n10. 22 (금),페퍼저축은행,0,:,3,GS칼텍스\n10. 23 (토),한국전력,3,:,1,KB손해보험\n10. 23 (토),KGC인삼공사,3,:,0,한국도로공사\n10. 24 (일),우리카드,1,:,3,현대캐피탈\n10. 24 (일),현대건설,3,:,1,흥국생명\n10. 26 (화),OK금융그룹,3,:,1,KB손해보험\n10. 26 (화),IBK기업은행,1,:,3,한국도로공사\n10. 27 (수),현대캐피탈,3,:,2,대한항공\n10. 27 (수),GS칼텍스,1,:,3,현대건설\n10. 28 (목),우리카드,3,:,0,한국전력\n10. 28 (목),KGC인삼공사,3,:,1,흥국생명\n10. 29 (금),KB손해보험,2,:,3,삼성화재\n10. 29 (금),한국도로공사,3,:,0,페퍼저축은행\n10. 30 (토),대한항공,3,:,0,OK금융그룹\n10. 30 (토),GS칼텍스,3,:,1,IBK기업은행\n10. 31 (일),한국전력,3,:,0,현대캐피탈\n10. 31 (일),KGC인삼공사,0,:,3,현대건설\n'

In [7]:
with open('배구승패.csv','w') as f:
    f.write(data_list)
f.close

<function TextIOWrapper.close()>

</br>
</br> 

In [11]:
url_list = []
for cnt in range(1,15):
    target_url = f'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=2022&ye=2022&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da={cnt}&o1=TPA&&de=1&lr=0&tr=&cv=&ml=1&sn=265&pa=0&si=&cn='
    url_list.append(target_url+'\n')
print(url_list)

['http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=2022&ye=2022&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=TPA&&de=1&lr=0&tr=&cv=&ml=1&sn=265&pa=0&si=&cn=\n', 'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=2022&ye=2022&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=2&o1=TPA&&de=1&lr=0&tr=&cv=&ml=1&sn=265&pa=0&si=&cn=\n', 'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=2022&ye=2022&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=3&o1=TPA&&de=1&lr=0&tr=&cv=&ml=1&sn=265&pa=0&si=&cn=\n', 'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=2022&ye=2022&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=4&o1=TPA&&de=1&lr=0&tr=&cv=&ml=1&sn=265&pa=0&si=&cn=\n', 'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=2022&ye=2022&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=5&o1=TPA&&de=1&lr=0&tr=&cv=&ml=1&sn=265&pa=0&si=&cn=\n', 'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=2022&ye=2022&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=6&o1=T